In [13]:
import os
import pandas as pd
from sqlalchemy import create_engine, text

import psycopg2
from psycopg2 import sql

In [15]:
table_creation_sql = '''

GRANT CREATE ON SCHEMA public TO prueba; ---solo  en caso de que lo necesites


CREATE TABLE IF NOT EXISTS city_day (
    City VARCHAR(500),
    Date DATE,  -- Cambié a DATE si solo necesitas la fecha sin la hora
    PM25 DOUBLE PRECISION,
    PM10 DOUBLE PRECISION,
    NO DOUBLE PRECISION,
    NO2 DOUBLE PRECISION,
    NOx DOUBLE PRECISION,
    NH3 DOUBLE PRECISION,
    CO DOUBLE PRECISION,
    SO2 DOUBLE PRECISION,
    O3 DOUBLE PRECISION,
    Benzene DOUBLE PRECISION,
    Toluene DOUBLE PRECISION,
    Xylene DOUBLE PRECISION,
    AQI DOUBLE PRECISION,
    AQI_Bucket VARCHAR(50)
);

CREATE TABLE IF NOT EXISTS  city_hour (
    City VARCHAR(500),
    Datetime TIMESTAMP,
    PM25 DOUBLE PRECISION,
    PM10 DOUBLE PRECISION,
    NO DOUBLE PRECISION,
    NO2 DOUBLE PRECISION,
    NOx DOUBLE PRECISION,
    NH3 DOUBLE PRECISION,
    CO DOUBLE PRECISION,
    SO2 DOUBLE PRECISION,
    O3 DOUBLE PRECISION,
    Benzene DOUBLE PRECISION,
    Toluene DOUBLE PRECISION,
    Xylene DOUBLE PRECISION,
    AQI DOUBLE PRECISION,
    AQI_Bucket VARCHAR(50)
);

CREATE TABLE IF NOT EXISTS  stations (
    StationId VARCHAR(50) PRIMARY KEY,
    StationName VARCHAR(100),
    City VARCHAR(100),
    State VARCHAR(100),
    Status VARCHAR(50)
);

CREATE TABLE IF NOT EXISTS  station_day (
    StationId VARCHAR(500),
    Date DATE,
    PM25 DOUBLE PRECISION,
    PM10 DOUBLE PRECISION,
    NO DOUBLE PRECISION,
    NO2 DOUBLE PRECISION,
    NOx DOUBLE PRECISION,
    NH3 DOUBLE PRECISION,
    CO DOUBLE PRECISION,
    SO2 DOUBLE PRECISION,
    O3 DOUBLE PRECISION,
    Benzene DOUBLE PRECISION,
    Toluene DOUBLE PRECISION,
    Xylene DOUBLE PRECISION,
    AQI DOUBLE PRECISION,
    AQI_Bucket VARCHAR(50)
);

CREATE TABLE IF NOT EXISTS  station_hour (
    StationId VARCHAR(500),
    Datetime TIMESTAMP,
    PM25 DOUBLE PRECISION,
    PM10 DOUBLE PRECISION,
    NO DOUBLE PRECISION,
    NO2 DOUBLE PRECISION,
    NOx DOUBLE PRECISION,
    NH3 DOUBLE PRECISION,
    CO DOUBLE PRECISION,
    SO2 DOUBLE PRECISION,
    O3 DOUBLE PRECISION,
    Benzene DOUBLE PRECISION,
    Toluene DOUBLE PRECISION,
    Xylene DOUBLE PRECISION,
    AQI DOUBLE PRECISION,
    AQI_Bucket VARCHAR(500)
);

'''

In [19]:

llaves = '''

ALTER TABLE city_day
ADD City_Date_Key VARCHAR(800);

UPDATE city_day
SET City_Date_Key = City || '_' || Date;


ALTER TABLE city_hour
ADD City_Datetime_Key VARCHAR(150);

UPDATE city_hour
SET City_Datetime_Key = City || '_' || Datetime;


ALTER TABLE station_day
ADD City_stationDate_Key VARCHAR(800);

UPDATE station_day
SET City_stationDate_Key = StationId || '_' || Date;


ALTER TABLE station_hour
ADD City_station_Datetime_Key VARCHAR(800);

UPDATE station_hour
SET City_station_Datetime_Key = StationId || '_' || Datetime;

'''



In [17]:
import os
import pandas as pd

# Ruta de la carpeta que contiene los archivos
folder_path = "raw_zone"

# Diccionario para almacenar los DataFrames
dataframes_dict = {}

# Recorrer todos los archivos en la carpeta
for file_name in os.listdir(folder_path):
    file_path = os.path.join(folder_path, file_name)
    
    # Verificar si es un archivo (y no una carpeta u otro tipo) y que tenga extensión .parquet
    if os.path.isfile(file_path) and file_name.endswith(".parquet"):
        print(f"Procesando archivo: {file_name}")
        
        try:
            # Leer archivo Parquet en un DataFrame
            df = pd.read_parquet(file_path)
            # Limpiar los nombres de las columnas
            df.columns = df.columns.str.replace(r'[\W_]+', '', regex=True)  # Eliminar caracteres especiales y espacios
            df.columns = df.columns.str.replace(' ', '')
            # Eliminar la extensión .parquet para usar solo el nombre base como clave
            base_name = os.path.splitext(file_name)[0]
            
            # Guardar el DataFrame en el diccionario
            dataframes_dict[base_name] = df
        
        except Exception as e:
            print(f"Error al procesar el archivo {file_name}: {e}")

# Opcional: Imprimir las claves del diccionario para verificar los archivos procesados
print("Archivos procesados:", dataframes_dict.keys())


Procesando archivo: city_day.parquet
Procesando archivo: city_hour.parquet
Procesando archivo: stations.parquet
Procesando archivo: station_day.parquet
Procesando archivo: station_hour.parquet
Archivos procesados: dict_keys(['city_day', 'city_hour', 'stations', 'station_day', 'station_hour'])


In [25]:
from sqlalchemy import create_engine, text
import pandas as pd

# Configuración de conexión a PostgreSQL
host = "localhost"
port = "5432"
database = "india"
usuario = "prueba"
contraseña = "1234"

# Crear el string de conexión
connection_string = f'postgresql://{usuario}:{contraseña}@{host}:{port}/{database}'
engine = create_engine(connection_string)
conn = engine.connect()


In [31]:
try:
    # Crear las tablas
    with engine.connect() as conn:
        conn.execute(text(table_creation_sql.strip()))
        conn.commit()
except Exception as e:
    print("Ocurrió un error al crear las tablas:", e)

In [ ]:

# Cargar los DataFrames en las tablas
try:
    for table_name, df in dataframes_dict.items():
        df.to_sql(table_name, engine, if_exists='replace', index=False)
except Exception as e:
    print("Ocurrió un error al cargar los DataFrames:", e)

In [27]:

# Agregar llaves a las tablas después de cargar los datos
try:
    with engine.connect() as conn:
        conn.execute(text(llaves.strip()))
        conn.commit()
except Exception as e:
    print("Ocurrió un error al agregar las llaves:", e)

finally:
    engine.dispose()


Ocurrió un error al crear las tablas: (psycopg2.errors.InsufficientPrivilege) permiso denegado al esquema public
LINE 1: CREATE TABLE IF NOT EXISTS city_day (
                                   ^

[SQL: CREATE TABLE IF NOT EXISTS city_day (
    City VARCHAR(500),
    Date DATE,  -- Cambié a DATE si solo necesitas la fecha sin la hora
    PM25 DOUBLE PRECISION,
    PM10 DOUBLE PRECISION,
    NO DOUBLE PRECISION,
    NO2 DOUBLE PRECISION,
    NOx DOUBLE PRECISION,
    NH3 DOUBLE PRECISION,
    CO DOUBLE PRECISION,
    SO2 DOUBLE PRECISION,
    O3 DOUBLE PRECISION,
    Benzene DOUBLE PRECISION,
    Toluene DOUBLE PRECISION,
    Xylene DOUBLE PRECISION,
    AQI DOUBLE PRECISION,
    AQI_Bucket VARCHAR(50)
);

CREATE TABLE IF NOT EXISTS  city_hour (
    City VARCHAR(500),
    Datetime TIMESTAMP,
    PM25 DOUBLE PRECISION,
    PM10 DOUBLE PRECISION,
    NO DOUBLE PRECISION,
    NO2 DOUBLE PRECISION,
    NOx DOUBLE PRECISION,
    NH3 DOUBLE PRECISION,
    CO DOUBLE PRECISION,
    SO2 DOUBLE P